# PROYECTO DE INTELIGENCIA ARTIFICIAL: Seleccion de características

Las siguientes funciones se encargan de obtener los atributos con los que trabajaremos, ademas de obtener los atributos ordenados y los objetivos para ambos csv.

Importaciones:

In [3]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

from sklearn.tree import DecisionTreeClassifier

from matplotlib import pyplot
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer

In [4]:
def lectura_titanic():
    fichero = pd.read_csv('titanic.csv')    
    return fichero

def lectura_breastCancer():
    fichero = pd.read_csv('BreastCancer.csv')
    return fichero

In [5]:
titanic = lectura_titanic()
breast_cancer = lectura_breastCancer()

In [6]:
# Eleccion de semilla inicial aleatoria.
np.random.seed(357823)

# Atributos discretos, continuos y objetivo de titanic.csv.
atributos_discretos_titanic = ['Sex', 'Embarked', 'Alone', 'Deck']
atributos_continuos_titanic = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Initial', 
                           'Age_band', 'Family_Size', 'Fare_cat', 'Title', 'Is_Married']
atributos_titanic = titanic.loc[:, atributos_discretos_titanic + atributos_continuos_titanic]
objetivo_titanic = titanic['Survived']

# Atributos discretos, continuos y objetivo de BreastCancer.csv.
atributos_continuos_breast_cancer = ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 
                           'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 
                           'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error',
                           'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 
                           'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points',
                           'worst symmetry', 'worst fractal dimension']
																				
atributos_breast_cancer = breast_cancer.loc[:, atributos_continuos_breast_cancer]
objetivo_breast_cancer = breast_cancer['diagnosis']

##### A continuación realizaremos los algoritmos de busqueda con los que vamos a trabajar. Para ello, trabajaremos con el algoritmo de "Decision tree clasifier" y el algoritmo de "Naives bayes".

#### Decision tree clasifier

#### Funcion de rendimiento

In [7]:
codificador_atributos_discretos = OrdinalEncoder()
# codificador_atributos_discretos.fit(atributos[atributos_discretos])

atributos = atributos_titanic
atributos_discretos = atributos_discretos_titanic
atributos[atributos_discretos] = codificador_atributos_discretos.fit_transform(atributos[atributos_discretos])

# codificador_objetivo = LabelEncoder()
# objetivo = codificador_objetivo.fit_transform(objetivo)

In [38]:
def rendimiento(atributos, objetivo, feature):
    
    # tree = decision_tree(atributos, objetivo, atributos_discretos)
    clasificador_CART = DecisionTreeClassifier()
    score = cross_validate(clasificador_CART, atributos_titanic[feature], objetivo_titanic, cv = 10, scoring = 'balanced_accuracy')
    return np.mean(score['test_score'])

In [39]:
# feature = ['Embarked', 'Pclass', 'Fare', 'Initial']
feature = ['Initial', 'SibSp', 'Deck', 'Fare_cat', 'Title']
rendimiento(atributos, objetivo_titanic, feature)

0.8153328806269983

In [46]:
def busqueda_secuencial_hacia_atras(atributos, objetivo, n_exp=10, cv=10):
    # Inicialización
    solucion_actual = list(atributos.columns)
    n_features = len(solucion_actual)
    resultados = []

     # Guardar el rendimiento de la solución inicial completa
    rendimiento_inicial = rendimiento(atributos, objetivo, solucion_actual)
    resultados.append((solucion_actual.copy(), len(solucion_actual), rendimiento_inicial))

    
    # Ejecución
    while len(solucion_actual) > 1:
        
        mejor_rendimiento = -float('inf')
        peor_feature = None
        
        # Evaluar todas las posibles soluciones temporales
        for feature in solucion_actual:
            solucion_temporal = solucion_actual.copy()
            solucion_temporal.remove(feature)
            rendimiento_temporal = rendimiento(atributos, objetivo, solucion_temporal)

            print(f"Evaluando sin {feature}: {solucion_temporal} - Rendimiento: {rendimiento_temporal}")

            # Guardar el rendimiento de la peor característica
            if rendimiento_temporal > mejor_rendimiento:
                mejor_rendimiento = rendimiento_temporal
                peor_feature = feature
                
        
        # Actualizar la solución actual eliminando la peor característica
        solucion_actual.remove(peor_feature)

        resultados.append((solucion_actual.copy(), len(solucion_actual), mejor_rendimiento))
    
    # Crear un DataFrame con los resultados
    df_resultados = pd.DataFrame(resultados, columns=['MejorSolucionTemporal', 'Tamaño', 'Rendimiento']).sort_values(by='Rendimiento', ascending=False)
    
    # Devolver los resultados en forma de DataFrame
    return df_resultados

In [47]:
busqueda_secuencial_hacia_atras(atributos, objetivo_titanic)

Evaluando sin Sex: ['Embarked', 'Alone', 'Deck', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Initial', 'Age_band', 'Family_Size', 'Fare_cat', 'Title', 'Is_Married'] - Rendimiento: 0.7701998981410746
Evaluando sin Embarked: ['Sex', 'Alone', 'Deck', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Initial', 'Age_band', 'Family_Size', 'Fare_cat', 'Title', 'Is_Married'] - Rendimiento: 0.7660937102113573
Evaluando sin Alone: ['Sex', 'Embarked', 'Deck', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Initial', 'Age_band', 'Family_Size', 'Fare_cat', 'Title', 'Is_Married'] - Rendimiento: 0.7734466513878278
Evaluando sin Deck: ['Sex', 'Embarked', 'Alone', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Initial', 'Age_band', 'Family_Size', 'Fare_cat', 'Title', 'Is_Married'] - Rendimiento: 0.7612755000990296
Evaluando sin Pclass: ['Sex', 'Embarked', 'Alone', 'Deck', 'Age', 'SibSp', 'Parch', 'Fare', 'Initial', 'Age_band', 'Family_Size', 'Fare_cat', 'Title', 'Is_Married'] - Rendimiento: 0.7818545397957163
Evalu

,MejorSolucionTemporal,Tamaño,Rendimiento
4,"[Sex, Alone, Deck, Pclass, SibSp, Parch, Fare,...",11,0.813812
8,"[Sex, Pclass, SibSp, Parch, Fare, Initial, Fam...",7,0.813623
10,"[Pclass, SibSp, Fare, Initial, Family_Size]",5,0.813062
7,"[Sex, Deck, Pclass, SibSp, Parch, Fare, Initia...",8,0.812325
3,"[Sex, Embarked, Alone, Deck, Pclass, SibSp, Pa...",12,0.812127
9,"[Sex, Pclass, SibSp, Fare, Initial, Family_Size]",6,0.811030
5,"[Sex, Deck, Pclass, SibSp, Parch, Fare, Initia...",10,0.810896
6,"[Sex, Deck, Pclass, SibSp, Parch, Fare, Initia...",9,0.810854
11,"[Pclass, SibSp, Fare, Initial]",4,0.806354
12,"[Pclass, Fare, Initial]",3,0.803975


In [67]:
def busqueda_secuencial_hacia_atras_mixta(atributos, objetivo, n_exp=10, cv=7, umbral=10):
    # Inicialización
    solucion_actual = list(atributos.columns)
    añadidos = []
    eliminados = []
    iteraciones_sin_mejora = 0
    rendimiento_actual = rendimiento(atributos, objetivo, solucion_actual)
    
    resultados = [(solucion_actual.copy(), len(solucion_actual), rendimiento_actual)]
    
    # Ejecución
    while len(eliminados) < len(atributos.columns):
        mejor_rendimiento = -float('inf')
        peor_feature = None
        
        # Evaluar todas las posibles soluciones temporales eliminando una variable
        for feature in solucion_actual:
            if feature not in eliminados:
                solucion_temporal = solucion_actual.copy()
                solucion_temporal.remove(feature)
                rendimiento_temporal = rendimiento(atributos, objetivo, solucion_temporal)

                if rendimiento_temporal > mejor_rendimiento:
                    mejor_rendimiento = rendimiento_temporal
                    peor_feature = feature
        
        # Actualizar la solución actual eliminando la peor característica
        solucion_actual.remove(peor_feature)
        eliminados.append(peor_feature)
        rendimiento_actual = mejor_rendimiento

        resultados.append((solucion_actual.copy(), len(solucion_actual), rendimiento_actual))

        # Intentar añadir la mejor característica si hay mejora
        mejor_rendimiento_añadir = -float('inf')
        mejor_feature_añadir = None
        
        for feature in atributos.columns:
            if feature not in solucion_actual and feature not in añadidos:
                solucion_temporal = solucion_actual.copy()
                solucion_temporal.append(feature)
                rendimiento_temporal = rendimiento(atributos, objetivo, solucion_temporal)

                if rendimiento_temporal > mejor_rendimiento_añadir:
                    mejor_rendimiento_añadir = rendimiento_temporal
                    mejor_feature_añadir = feature

        if mejor_rendimiento_añadir > rendimiento_actual:
            solucion_actual.append(mejor_feature_añadir)
            añadidos.append(mejor_feature_añadir)
            rendimiento_actual = mejor_rendimiento_añadir
            iteraciones_sin_mejora = 0
        else:
            iteraciones_sin_mejora += 1

        resultados.append((solucion_actual.copy(), len(solucion_actual), rendimiento_actual))

        # Evaluar condición de parada
        if iteraciones_sin_mejora >= umbral:
            break
    
    # Crear un DataFrame con los resultados
    df_resultados = pd.DataFrame(resultados, columns=['MejorSolucionTemporal', 'Tamaño', 'Rendimiento'])

    # Filtrar para que solo quede el mejor rendimiento por cada tamaño de conjunto
    df_resultados = df_resultados.sort_values(by=['Tamaño', 'Rendimiento'], ascending=[True, False])
    df_resultados = df_resultados.drop_duplicates(subset=['Tamaño'], keep='first')
    
    # Devolver los resultados en forma de DataFrame
    return df_resultados.sort_values(by=['Rendimiento'], ascending=False)

In [68]:
busqueda_secuencial_hacia_atras_mixta(atributos, objetivo_titanic)

,MejorSolucionTemporal,Tamaño,Rendimiento
22,"[Pclass, SibSp, Fare, Initial, Fare_cat, Famil...",7,0.813623
19,"[Sex, Pclass, SibSp, Fare, Initial, Fare_cat]",6,0.812023
14,"[Sex, Embarked, Deck, Pclass, SibSp, Fare, Ini...",9,0.810657
7,"[Sex, Embarked, Deck, Pclass, SibSp, Parch, Fa...",11,0.810163
15,"[Sex, Embarked, Pclass, SibSp, Fare, Initial, ...",8,0.808998
5,"[Sex, Embarked, Deck, Pclass, SibSp, Parch, Fa...",12,0.808684
9,"[Sex, Embarked, Deck, Pclass, SibSp, Fare, Ini...",10,0.807179
3,"[Sex, Embarked, Alone, Deck, Pclass, SibSp, Pa...",13,0.804272
1,"[Sex, Embarked, Alone, Deck, Pclass, SibSp, Pa...",14,0.782680
0,"[Sex, Embarked, Alone, Deck, Pclass, Age, SibS...",15,0.782031
